In [1]:
%matplotlib widget
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import re
import peakutils
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import scipy.signal

In [2]:
from scipy import sparse
from scipy.sparse.linalg import spsolve

def baseline_als(y, lam, p, niter=10):
    """Функция для сглаживания спектра методом Asymmetric Least Squares Smoothing"""
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

In [3]:
df = pd.read_csv("datasets/earLobe.csv")
#df.head()

In [4]:
# df_earLobe = pd.read_csv("datasets/earLobe.csv")
# df_innerArm = pd.read_csv("datasets/innerArm.csv")
# df_thumbNail = pd.read_csv("datasets/thumbNail.csv")
# df_vein = pd.read_csv("datasets/vein.csv")
# df = df_earLobe.append([df_innerArm.drop(0, axis=0), df_thumbNail.drop(0, axis=0), df_vein.drop(0, axis=0)])
# df.index = range(0, 81)

In [5]:
has_DM2 = df.pop('has_DM2')
patientID = df.pop('patientID')

In [6]:
df.head()

,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,...,Var3152,Var3153,Var3154,Var3155,Var3156,Var3157,Var3158,Var3159,Var3160,Var3161
0,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159
1,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,181.800000,...,0,0,0,0,0,0,0,0,0,0
2,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,162.800000,...,0,0,0,0,0,0,0,0,0,0
3,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,107.400000,...,0,0,0,0,0,0,0,0,0,0
4,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,290.166667,...,0,0,0,0,0,0,0,0,0,0


In [7]:
plt.plot(df.iloc[0], df.iloc[1]);

FigureCanvasNbAgg()

#### Crop Data to 800-1800 cm^-1 

In [8]:
droped_columns = []
for col in df.columns:
    if int(re.findall(r'\d+', col)[0]) <= 800 or int(re.findall(r'\d+', col)[0]) >= 1800:
        droped_columns.append(col)

In [9]:
df.drop(droped_columns, axis=1, inplace=True)

In [10]:
df.head()

,Var801,Var802,Var803,Var804,Var805,Var806,Var807,Var808,Var809,Var810,...,Var1790,Var1791,Var1792,Var1793,Var1794,Var1795,Var1796,Var1797,Var1798,Var1799
0,799.000000,800.000000,801.000000,802.000000,803.000000,804.000000,805.000000,806.000000,807.000000,808.000000,...,1788.000000,1789.000000,1790.000000,1791.000000,1792.000000,1793.000000,1794.000000,1795.000000,1796.000000,1797.000000
1,187.887152,191.185864,195.943112,197.019885,194.774564,192.743616,191.215888,190.194934,195.590097,200.985260,...,51.245700,54.463985,56.393459,49.932257,47.622393,48.354281,51.604005,53.379442,51.303317,50.599999
2,169.308038,175.024512,183.282382,189.843460,194.872977,195.259000,184.744665,175.342270,180.018082,184.693894,...,58.028577,56.525036,54.495767,52.179488,49.228832,45.813356,48.117433,51.241115,55.393367,55.345582
3,120.285950,120.814867,122.161259,124.735393,128.417717,129.147617,122.946858,117.390883,119.998546,122.606209,...,27.668530,35.821674,43.497690,39.352768,34.854444,30.097175,28.998998,29.130067,31.572569,35.551240
4,305.211868,307.784993,313.469754,317.818830,320.962275,321.376920,315.385852,310.069199,313.291306,316.513413,...,65.438088,65.602856,66.066346,66.371121,67.086658,68.103169,66.164164,65.640090,68.896758,68.855319


In [11]:
# %matplotlib inline
# from matplotlib import pyplot as plt

plt.plot(df.iloc[0], df.iloc[1]);

#### Удаляем cosmic rays

In [12]:
columns = list(df.columns)
for row in range(1,21):
    for n,value in enumerate(columns):
        if df[columns[n]][row] < 0 or df[columns[n]][row] > 500:
            df[columns[n]][row] = df[columns[n-1]][row]

In [13]:
plt.plot(df.iloc[0], df.iloc[10]);

#### Делаем коррекцию фона

In [14]:
# Baseline removal
y1 = df.iloc[10].values
baseline_values = peakutils.baseline(y1)

plt.plot(y1);
plt.plot(baseline_values);
plt.plot(y1 - baseline_values);

In [15]:
# Применяем коррекцию фону ко всему датасету

for row in range(1,21):
    baseline_values = peakutils.baseline(df.iloc[row].values)
    df.iloc[row] = df.iloc[row].values - baseline_values


#### Сглаживание спектра

In [16]:
# Smoothing with Asymmetric Least Squares
p = 0.01
lam = 10
y1 = df.iloc[10].values
y2 = baseline_als(y1, lam, p, niter=10)

plt.plot(y1);
plt.plot(y2);

In [17]:
# Smoothing with a Savitzky-Golay filter
y_noise = df.iloc[10].values
y_sg = scipy.signal.savgol_filter(y_noise, 31, 5)
plt.plot(y_noise);
plt.plot(y_sg);

In [18]:
# Применяем сглаживание ко всему датасету:

for row in range(1,21):
    #df.iloc[row] = baseline_als(df.iloc[row], lam, p, niter=10)
    df.iloc[row] = scipy.signal.savgol_filter(df.iloc[row], 31, 5)
    

#### Разделяем данные на матрицу с признаками X и на столбец с целевой переменной (метками)
И удаляем первую строку с частотами

In [19]:
X, y = df.drop(0), has_DM2.drop(0)

#### Оценку качества модели (accuracy)  будем проводить методом отложенной выборки (hold-out set)
При таком подходе мы оставляем какую-то долю обучающей выборки (как правило от 20% до 40%), обучаем модель на остальных данных (60-80% исходной выборки) и считаем некоторую метрику качества модели (например, самое простое – долю правильных ответов в задаче классификации) на отложенной выборке.

In [20]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=.3, random_state=7)

In [21]:
X_train.shape

(14, 999)

In [22]:
y.value_counts(normalize=True)

1.0    0.55
0.0    0.45
Name: has_DM2, dtype: float64

#### Точность предсказания должна быть лучше как минимум чем 55%

#### Centering data to zero-mean

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_holdout = scaler.transform(X_holdout)

In [24]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)  
X_train = pca.fit_transform(X_train)  
X_holdout  = pca.transform(X_holdout)  

In [25]:
# pca.explained_variance_ratio_ 

In [26]:
X_train.shape

(14, 2)

### Сохраняем данные в pickle-файл

In [27]:
import pickle

In [28]:
with open('X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f)
with open('y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)

In [29]:
with open('X_holdout.pickle', 'wb') as f:
    pickle.dump(X_holdout, f)
with open('y_holdout.pickle', 'wb') as f:
    pickle.dump(y_holdout, f)